In [38]:
import pandas as pd
import os
from glob import glob
from reports import get_average_classification_report
pd.options.display.float_format = '{:.4f}'.format
from functools import reduce
pd.options.display.float_format = '{:.4f}'.format

In [27]:
target_trained_with_nums = [400, 500, 600, 700, 800, 900, 1000, 2000]
dirnames = sorted(glob(os.path.join('C:\DATA\ComBERT', 'results_spamFiltering', 'CM_epochs', '*_seed0')))
dirnames

['C:\\DATA\\ComBERT\\results_spamFiltering\\CM_epochs\\Araci_CM_epoch3_seed0',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\CM_epochs\\BERT_CM_epoch3_seed0',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\CM_epochs\\SECBERT_CM_epoch3_seed0',
 'C:\\DATA\\ComBERT\\results_spamFiltering\\CM_epochs\\Yang_CM_epoch3_seed0']

In [57]:
dfs = []
for dirname in dirnames:
    model_name, method_name, epoch_num, seed_num = os.path.basename(dirname).split('_')
    
    records = []
    for target_trained_with_num in target_trained_with_nums:
        filepaths = glob(os.path.join(os.path.dirname(dirname), '{}_*'.format('_'.join([model_name, method_name, epoch_num])), \
                  'classification_report_train_{}.csv'.format(target_trained_with_num)))
        
        column_name = '_'.join([model_name, method_name, epoch_num])
        averaged_classification_report_df = get_average_classification_report([pd.read_csv(filepath) for filepath in filepaths])
        acc = 100*averaged_classification_report_df.filter(items = ['accuracy'], axis=0)['f1-score'].values[0]
        records.append((target_trained_with_num, acc))
    dfs.append(pd.DataFrame(records, columns=['trained_with', column_name]).sort_values(by=['trained_with']))

result = reduce(lambda df1,df2: pd.merge(df1,df2,on='trained_with'), dfs)
result.set_index('trained_with', inplace=True)
result.loc['Average'] = result.mean()

result

,Araci_CM_epoch3,BERT_CM_epoch3,SECBERT_CM_epoch3,Yang_CM_epoch3
trained_with,,,,
400,59.3600,60.1033,56.1100,66.3567
500,67.3667,64.4700,62.8433,70.6500
600,69.8733,68.2367,67.6500,72.2333
700,71.5367,69.8400,70.5433,75.0533
800,72.6033,67.9633,69.6900,76.5500
900,74.5300,73.0967,73.6567,77.9167
1000,75.9600,72.8667,74.0600,78.3767
2000,80.9300,80.4200,80.1467,80.9967
Average,71.5200,69.6246,69.3375,74.7667
